In [14]:
"""
BOPTEST HVAC 离散控制 - 完整训练版本
- 4个控制变量：风扇、送风温度、加热设定点、冷却设定点
- 1000 episodes 训练，每50次评估
- 每episode仿真7天
- 详细的action和环境状态打印
"""

import sys
import requests
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import time
from datetime import datetime
import matplotlib.pyplot as plt
import os

# =====================================================================
# 配置参数
# =====================================================================
BOPTEST_URL = "http://localhost"
TESTCASE_NAME = "bestest_air"

# 训练参数
TOTAL_EPISODES = 1000        # 总训练episodes
EVAL_FREQUENCY = 50          # 每50个episode评估一次
EVAL_EPISODES = 3            # 评估时运行3个episode
STEPS_PER_EPISODE = 672      # 7天 * 96步/天 = 672步 (15分钟/步)
STEP_PERIOD = 900            # 15分钟 = 900秒

# =====================================================================
# 动作空间定义 - 4个控制变量
# =====================================================================
FAN_LEVELS = [0.3, 0.6, 1.0]                    # 风扇: 低/中/高
SUPPLY_TEMP_LEVELS = [288.15, 298.15, 308.15]   # 送风: 15°C / 25°C / 35°C  
HEAT_SETPOINT_LEVELS = [291.15, 293.15, 295.15] # 加热设定: 18°C / 20°C / 22°C
COOL_SETPOINT_LEVELS = [296.15, 298.15, 300.15] # 冷却设定: 23°C / 25°C / 27°C

NUM_ACTIONS = len(FAN_LEVELS) * len(SUPPLY_TEMP_LEVELS) * len(HEAT_SETPOINT_LEVELS) * len(COOL_SETPOINT_LEVELS)

# =====================================================================
# 工具函数
# =====================================================================

def log(msg, level="INFO"):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {level:5s} | {msg}")

def K2C(k):
    """开尔文转摄氏度"""
    return k - 273.15

def action_to_values(action_idx):
    """动作索引 → 控制值"""
    cool_idx = action_idx % len(COOL_SETPOINT_LEVELS)
    remaining = action_idx // len(COOL_SETPOINT_LEVELS)
    heat_idx = remaining % len(HEAT_SETPOINT_LEVELS)
    remaining = remaining // len(HEAT_SETPOINT_LEVELS)
    supply_idx = remaining % len(SUPPLY_TEMP_LEVELS)
    fan_idx = remaining // len(SUPPLY_TEMP_LEVELS)
    
    return {
        'fan': FAN_LEVELS[fan_idx],
        'supply_temp': SUPPLY_TEMP_LEVELS[supply_idx],
        'heat_setpoint': HEAT_SETPOINT_LEVELS[heat_idx],
        'cool_setpoint': COOL_SETPOINT_LEVELS[cool_idx]
    }

def action_to_string(action_idx):
    """动作索引 → 可读字符串"""
    v = action_to_values(action_idx)
    return (f"Fan={v['fan']:.1f} | "
            f"供风={K2C(v['supply_temp']):.0f}°C | "
            f"加热={K2C(v['heat_setpoint']):.0f}°C | "
            f"冷却={K2C(v['cool_setpoint']):.0f}°C")

# =====================================================================
# 环境类
# =====================================================================

class BOPTESTEnv:
    def __init__(self):
        self.url = BOPTEST_URL
        self.testid = None
        self.obs_dim = 8
        self.action_dim = NUM_ACTIONS
        
    def _select_testcase(self):
        """选择测试案例"""
        log(f"选择测试案例: {TESTCASE_NAME}")
        try:
            resp = requests.post(f"{self.url}/testcases/{TESTCASE_NAME}/select", timeout=120)
            if resp.status_code == 200:
                self.testid = resp.json().get('testid')
                log(f"Test ID: {self.testid}")
                time.sleep(8)
                return True
        except Exception as e:
            log(f"选择失败: {e}", "ERROR")
        return False
    
    def reset(self, start_time=0):
        """重置环境"""
        if not self.testid and not self._select_testcase():
            raise Exception("无法选择测试案例")
        
        # 初始化
        requests.put(f"{self.url}/initialize/{self.testid}",
                    json={"start_time": start_time, "warmup_period": 0}, timeout=30)
        
        # 设置步长
        requests.put(f"{self.url}/step/{self.testid}",
                    json={"step": STEP_PERIOD}, timeout=10)
        
        # 获取初始观测
        resp = requests.post(f"{self.url}/advance/{self.testid}", json={}, timeout=30)
        payload = resp.json().get('payload', {})
        
        return self._get_obs(payload), payload
    
    def step(self, action_idx):
        """执行动作"""
        action = action_to_values(action_idx)
        
        # 确保 加热设定点 < 冷却设定点
        if action['heat_setpoint'] >= action['cool_setpoint']:
            action['cool_setpoint'] = action['heat_setpoint'] + 2.0
        
        control = {
            'fcu_oveFan_u': action['fan'],
            'fcu_oveFan_activate': 1,
            'fcu_oveTSup_u': action['supply_temp'],
            'fcu_oveTSup_activate': 1,
            'con_oveTSetHea_u': action['heat_setpoint'],
            'con_oveTSetHea_activate': 1,
            'con_oveTSetCoo_u': action['cool_setpoint'],
            'con_oveTSetCoo_activate': 1,
        }
        
        resp = requests.post(f"{self.url}/advance/{self.testid}", json=control, timeout=30)
        payload = resp.json().get('payload', {}) or {}
        
        obs = self._get_obs(payload)
        reward, reward_info = self._calc_reward(obs, action, payload)
        
        sim_time = payload.get('time', 0)
        done = sim_time >= 86400 * 7  # 7天结束
        
        info = {
            'sim_time': sim_time,
            'sim_day': sim_time / 86400,
            'sim_hour': (sim_time % 86400) / 3600,
            'room_temp': K2C(obs[0]),
            'outdoor_temp': K2C(obs[1]),
            'heating_power': payload.get('fcu_reaPHea_y', 0),
            'cooling_power': payload.get('fcu_reaPCoo_y', 0),
            'fan_power': payload.get('fcu_reaPFan_y', 0),
            'reward_info': reward_info
        }
        
        return obs, reward, done, info
    
    def _get_obs(self, payload):
        """提取观测"""
        if not payload:
            return np.zeros(self.obs_dim, dtype=np.float32)
        
        sim_time = payload.get('time', 0)
        hour = (sim_time % 86400) / 3600
        
        return np.array([
            payload.get('zon_reaTRooAir_y', 293.15),           # 室温 K
            payload.get('zon_weaSta_reaWeaTDryBul_y', 283.15), # 室外温度 K
            payload.get('zon_weaSta_reaWeaRelHum_y', 0.5),     # 湿度
            payload.get('fcu_reaPHea_y', 0) / 10000,           # 加热功率 (归一化)
            payload.get('fcu_reaPCoo_y', 0) / 10000,           # 冷却功率 (归一化)
            payload.get('fcu_reaPFan_y', 0) / 1000,            # 风扇功率 (归一化)
            np.sin(2 * np.pi * hour / 24),                     # 时间特征
            np.cos(2 * np.pi * hour / 24),
        ], dtype=np.float32)
    
    def _calc_reward(self, obs, action, payload):
        """
        计算奖励
        
        奖励 = 温度舒适度 + 能耗惩罚
        
        温度舒适度:
          - 室温在 [20, 24]°C 内: 奖励 = 5 - |室温 - 22|
          - 室温 < 20°C: 奖励 = -2 × (20 - 室温)  
          - 室温 > 24°C: 奖励 = -2 × (室温 - 24)
        
        能耗惩罚:
          - penalty = -0.3 × (加热功率 + 冷却功率 + 0.1×风扇功率)
        """
        room_temp_c = K2C(obs[0])
        
        # === 温度舒适度 ===
        COMFORT_MIN = 20.0
        COMFORT_MAX = 24.0
        COMFORT_TARGET = 22.0
        
        if COMFORT_MIN <= room_temp_c <= COMFORT_MAX:
            # 舒适范围内
            temp_reward = 5.0 - abs(room_temp_c - COMFORT_TARGET)
        elif room_temp_c < COMFORT_MIN:
            # 太冷
            temp_reward = -2.0 * (COMFORT_MIN - room_temp_c)
        else:
            # 太热
            temp_reward = -2.0 * (room_temp_c - COMFORT_MAX)
        
        # === 能耗惩罚 ===
        heat_power = obs[3]   # 已归一化
        cool_power = obs[4]
        fan_power = obs[5]
        energy_penalty = -0.3 * (heat_power + cool_power + 0.1 * fan_power)
        
        # === 总奖励 ===
        reward = temp_reward + energy_penalty
        
        reward_info = {
            'temp_reward': temp_reward,
            'energy_penalty': energy_penalty,
            'room_temp': room_temp_c
        }
        
        return reward, reward_info
    
    def stop(self):
        if self.testid:
            try:
                requests.put(f"{self.url}/stop/{self.testid}", timeout=10)
                self.testid = None
            except:
                pass

# =====================================================================
# A2C 网络和智能体
# =====================================================================

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden=256):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )
        self.actor = nn.Linear(hidden, action_dim)
        self.critic = nn.Linear(hidden, 1)
        
    def forward(self, x):
        feat = self.shared(x)
        return torch.softmax(self.actor(feat), dim=-1), self.critic(feat)

class A2CAgent:
    def __init__(self, state_dim, action_dim, lr=3e-4, gamma=0.99, entropy_coef=0.01):
        self.model = ActorCritic(state_dim, action_dim)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        self.entropy_coef = entropy_coef
        
    def select_action(self, state, explore=True):
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0)
            probs, value = self.model(state_t)
            
            if explore:
                action = torch.multinomial(probs, 1).item()
            else:
                action = probs.argmax().item()
            
            return action, probs[0].numpy(), value.item()
    
    def update(self, batch):
        states = torch.FloatTensor(np.array(batch['states']))
        actions = torch.LongTensor(batch['actions'])
        rewards = torch.FloatTensor(batch['rewards'])
        next_states = torch.FloatTensor(np.array(batch['next_states']))
        dones = torch.FloatTensor(batch['dones'])
        
        probs, values = self.model(states)
        _, next_values = self.model(next_states)
        
        # TD目标和优势
        td_target = rewards + self.gamma * next_values.squeeze() * (1 - dones)
        advantages = (td_target - values.squeeze()).detach()
        
        # 标准化优势
        if len(advantages) > 1:
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # 损失
        dist = torch.distributions.Categorical(probs)
        log_probs = dist.log_prob(actions)
        actor_loss = -(log_probs * advantages).mean()
        critic_loss = nn.functional.mse_loss(values.squeeze(), td_target.detach())
        entropy = dist.entropy().mean()
        
        loss = actor_loss + 0.5 * critic_loss - self.entropy_coef * entropy
        
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)
        self.optimizer.step()
        
        return {'loss': loss.item(), 'entropy': entropy.item()}
    
    def save(self, path):
        torch.save(self.model.state_dict(), path)
    
    def load(self, path):
        self.model.load_state_dict(torch.load(path))

# =====================================================================
# 训练监控
# =====================================================================

class Monitor:
    def __init__(self, save_dir="./output"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        
        self.train_rewards = []
        self.train_temps = []
        self.eval_rewards = []
        self.eval_temps = []
        self.eval_episodes = []
        
    def log_train(self, reward, temp):
        self.train_rewards.append(reward)
        self.train_temps.append(temp)
    
    def log_eval(self, episode, reward, temp):
        self.eval_episodes.append(episode)
        self.eval_rewards.append(reward)
        self.eval_temps.append(temp)
    
    def plot(self, filename="training_curves.png"):
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # 训练奖励
        ax = axes[0, 0]
        ax.plot(self.train_rewards, 'b-', alpha=0.3, label='Raw')
        if len(self.train_rewards) > 10:
            window = 10
            ma = np.convolve(self.train_rewards, np.ones(window)/window, mode='valid')
            ax.plot(range(window-1, len(self.train_rewards)), ma, 'b-', linewidth=2, label='MA-10')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Reward')
        ax.set_title('Training Episode Rewards')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 训练温度
        ax = axes[0, 1]
        ax.plot(self.train_temps, 'g-', alpha=0.3)
        ax.axhline(22, color='r', linestyle='--', label='Target')
        ax.axhspan(20, 24, alpha=0.1, color='green')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Avg Temperature (°C)')
        ax.set_title('Training Avg Room Temperature')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 评估奖励
        ax = axes[1, 0]
        if self.eval_rewards:
            ax.plot(self.eval_episodes, self.eval_rewards, 'ro-', linewidth=2, markersize=8)
            ax.set_xlabel('Episode')
            ax.set_ylabel('Eval Reward')
            ax.set_title('Evaluation Rewards')
            ax.grid(True, alpha=0.3)
        
        # 评估温度
        ax = axes[1, 1]
        if self.eval_temps:
            ax.plot(self.eval_episodes, self.eval_temps, 'go-', linewidth=2, markersize=8)
            ax.axhline(22, color='r', linestyle='--')
            ax.axhspan(20, 24, alpha=0.1, color='green')
            ax.set_xlabel('Episode')
            ax.set_ylabel('Eval Avg Temperature (°C)')
            ax.set_title('Evaluation Avg Room Temperature')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(self.save_dir, filename), dpi=150)
        plt.close()

# =====================================================================
# 打印函数
# =====================================================================

def print_step(step, info, action_idx, reward):
    """打印每步信息"""
    action_str = action_to_string(action_idx)
    print(f"  Step {step:4d} | "
          f"Day {info['sim_day']:.1f} Hour {info['sim_hour']:5.1f} | "
          f"室温={info['room_temp']:5.1f}°C | "
          f"室外={info['outdoor_temp']:5.1f}°C | "
          f"加热={info['heating_power']:7.0f}W | "
          f"冷却={info['cooling_power']:7.0f}W | "
          f"R={reward:+6.2f} | "
          f"Action: {action_str}")

def print_episode_summary(episode, total_reward, avg_temp, steps, mode="TRAIN"):
    """打印episode总结"""
    print(f"\n{'─'*80}")
    print(f"[{mode}] Episode {episode} 完成")
    print(f"  总奖励: {total_reward:8.1f}")
    print(f"  平均室温: {avg_temp:6.1f}°C (目标: 22°C, 舒适区: 20-24°C)")
    print(f"  步数: {steps}")
    print(f"{'─'*80}\n")

# =====================================================================
# 训练和评估函数
# =====================================================================

def run_episode(env, agent, explore=True, verbose=True, print_every=50):
    """
    运行一个episode
    
    Args:
        explore: 是否探索（训练时True，评估时False）
        verbose: 是否打印详细信息
        print_every: 每多少步打印一次
    """
    state, _ = env.reset()
    
    total_reward = 0
    temps = []
    trajectory = []
    
    for step in range(STEPS_PER_EPISODE):
        action, probs, value = agent.select_action(state, explore=explore)
        next_state, reward, done, info = env.step(action)
        
        trajectory.append({
            'state': state,
            'action': action,
            'reward': reward,
            'next_state': next_state,
            'done': done
        })
        
        total_reward += reward
        temps.append(info['room_temp'])
        
        # 打印
        if verbose and (step + 1) % print_every == 0:
            print_step(step + 1, info, action, reward)
        
        state = next_state
        
        if done:
            break
    
    return total_reward, np.mean(temps), trajectory

def evaluate(env, agent, num_episodes=3):
    """评估智能体"""
    print("\n" + "="*80)
    print("🔍 开始评估...")
    print("="*80)
    
    total_rewards = []
    total_temps = []
    
    for ep in range(num_episodes):
        log(f"评估 Episode {ep+1}/{num_episodes}")
        reward, avg_temp, _ = run_episode(env, agent, explore=False, verbose=True, print_every=100)
        total_rewards.append(reward)
        total_temps.append(avg_temp)
        print_episode_summary(ep+1, reward, avg_temp, STEPS_PER_EPISODE, mode="EVAL")
    
    avg_reward = np.mean(total_rewards)
    avg_temp = np.mean(total_temps)
    
    print(f"\n📊 评估结果: 平均奖励={avg_reward:.1f}, 平均温度={avg_temp:.1f}°C\n")
    
    return avg_reward, avg_temp

def train():
    """主训练循环"""
    
    # 检查连接
    try:
        resp = requests.get(f"{BOPTEST_URL}/testcases", timeout=5)
        if resp.status_code != 200:
            log("无法连接BOPTEST!", "ERROR")
            return
    except:
        log("无法连接BOPTEST!", "ERROR")
        return
    
    log("BOPTEST连接成功!")
    
    # 初始化
    env = BOPTESTEnv()
    agent = A2CAgent(state_dim=env.obs_dim, action_dim=env.action_dim)
    monitor = Monitor()
    
    # 经验缓冲区
    buffer = {k: deque(maxlen=50000) for k in ['states', 'actions', 'rewards', 'next_states', 'dones']}
    
    print("\n" + "="*80)
    print("🚀 开始训练")
    print(f"   总Episodes: {TOTAL_EPISODES}")
    print(f"   每Episode步数: {STEPS_PER_EPISODE} (7天仿真)")
    print(f"   评估频率: 每{EVAL_FREQUENCY}个episode")
    print(f"   动作空间: {NUM_ACTIONS}个离散动作")
    print("="*80 + "\n")
    
    best_eval_reward = float('-inf')
    
    for episode in range(1, TOTAL_EPISODES + 1):
        print(f"\n{'='*80}")
        print(f"📊 训练 Episode {episode}/{TOTAL_EPISODES}")
        print(f"{'='*80}")
        
        try:
            # 运行episode（训练时每50步打印一次）
            total_reward, avg_temp, trajectory = run_episode(
                env, agent, 
                explore=True, 
                verbose=True, 
                print_every=50
            )
        except Exception as e:
            log(f"Episode失败: {e}", "ERROR")
            env.stop()
            env.testid = None
            time.sleep(2)
            continue
        
        # 存储经验
        for t in trajectory:
            buffer['states'].append(t['state'])
            buffer['actions'].append(t['action'])
            buffer['rewards'].append(t['reward'])
            buffer['next_states'].append(t['next_state'])
            buffer['dones'].append(float(t['done']))
        
        # 更新模型
        if len(buffer['states']) >= 64:
            for _ in range(10):  # 每episode更新10次
                indices = np.random.choice(len(buffer['states']), 64, replace=False)
                batch = {k: [buffer[k][i] for i in indices] for k in buffer.keys()}
                agent.update(batch)
        
        # 记录
        monitor.log_train(total_reward, avg_temp)
        print_episode_summary(episode, total_reward, avg_temp, len(trajectory), mode="TRAIN")
        
        # 停止环境
        env.stop()
        
        # ============ 评估 ============
        if episode % EVAL_FREQUENCY == 0:
            eval_reward, eval_temp = evaluate(env, agent, num_episodes=EVAL_EPISODES)
            monitor.log_eval(episode, eval_reward, eval_temp)
            
            # 保存最佳模型
            if eval_reward > best_eval_reward:
                best_eval_reward = eval_reward
                agent.save(os.path.join(monitor.save_dir, "best_model.pth"))
                log(f"💾 新的最佳模型! 评估奖励: {best_eval_reward:.1f}")
            
            # 绘制曲线
            monitor.plot()
            log(f"📊 训练曲线已更新")
            
            env.stop()
        
        time.sleep(0.5)
    
    # 最终保存
    agent.save(os.path.join(monitor.save_dir, "final_model.pth"))
    monitor.plot("final_curves.png")
    
    print("\n" + "="*80)
    print("🎉 训练完成!")
    print(f"   总Episodes: {len(monitor.train_rewards)}")
    print(f"   最佳评估奖励: {best_eval_reward:.1f}")
    print(f"   输出目录: {monitor.save_dir}")
    print("="*80 + "\n")

# =====================================================================
# 主函数
# =====================================================================

if __name__ == "__main__":
    if len(sys.argv) > 1 and sys.argv[1] == "test":
        # 测试连接
        try:
            resp = requests.get(f"{BOPTEST_URL}/testcases", timeout=5)
            print(f"连接成功! 状态码: {resp.status_code}")
        except Exception as e:
            print(f"连接失败: {e}")
    else:
        train()

[18:46:28] INFO  | BOPTEST连接成功!

🚀 开始训练
   总Episodes: 1000
   每Episode步数: 672 (7天仿真)
   评估频率: 每50个episode
   动作空间: 81个离散动作


📊 训练 Episode 1/1000
[18:46:28] INFO  | 选择测试案例: bestest_air


KeyboardInterrupt: 